In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
import re

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
LAYER_COUNT = 4
LSTM_DIM = 256
TEXT_CLEANING_RE = "[\t\n\r_`]"
TEMPERATURE = 0.8

In [9]:

with open("./data/complete_lotr_precleaned.txt", "rb") as f:
    text = f.read().decode(encoding='utf-8')

text = re.sub(" +"," ",text.lower())
text = re.sub(TEXT_CLEANING_RE,"", text)    
vocab = sorted(set(text))
vocab_size = len(vocab)
print(vocab_size)
char_to_idx = {c:i for i,c in enumerate(vocab)}
idx_to_char = {i:c for c,i in char_to_idx.items()}


63


In [10]:
filepath = "model_weights_saved.hdf5"

testing_model = tf.keras.models.Sequential()
for i in range(LAYER_COUNT):
    testing_model.add(
            LSTM(
                LSTM_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                batch_input_shape=(1, 1, vocab_size),
                stateful=True
            )
        )
testing_model.add(Dense(vocab_size, activation = 'softmax'))
adam = tf.keras.optimizers.Adam(lr = 0.01)
testing_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

testing_model.load_weights(filepath)    

In [11]:
def gumbel_sample(probs, temperature = TEMPERATURE):
  """Helper function to sample an index from a probability array"""
  # from fchollet/keras
  probs = np.asarray(probs).astype('float64')
  probs = np.log(probs) / temperature
  exp_preds = np.exp(probs)
  probs = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, probs, 1)
  return np.argmax(probas)

def predict_next_char(model, current_char):
  x = np.zeros((1, 1, vocab_size))
  x[:,:,char_to_idx[current_char]] = 1
  y = model.predict(x, batch_size=1)
  next_char_idx = gumbel_sample(y[0,:])
  next_char = idx_to_char[next_char_idx]
  return next_char

def generate_text(model, seed, length):
    """Generate characters from a given seed"""
    generated_text = seed
    model.reset_states()
    for c in seed[:-1]:
        next_char = predict_next_char(model, c)
    current_char = seed[-1]

    for i in range(length - len(seed)):
        next_char = predict_next_char(model, current_char)
        current_char = next_char
        generated_text += current_char
    print(generated_text)


In [15]:
#generate_text(testing_model, seed = "The ring", length = 400)
gen_text_length = 200
seed_list = ["a","c","b","g","f","s"]
for starting in seed_list:
    for temperature in [.1,.8,1.0]:
        print(f"Temperature: {temperature} \n")
        generate_text(testing_model, seed = starting.lower(), length = gen_text_length)
        print("\n")

        

Temperature: 0.1 

ane his way the land, in his side named. he saw before was great and dead vain there came to the hills of the hills of shadow to waited forth the world on himself to get out of bearer, there is not fa


Temperature: 0.8 

ane the days to come. it was spicions, and all kings and a mean dreads were grim and tarily and then he said: 'ghost that it were many!' said parting. 'well, but the bitter for the lord, and he did no


Temperature: 1.0 

aning the great winter, and he was a land; but and precious had grew at the numbering. in the stream and recuising the last was on the first release whire was ruled upon the mountain. the dúnch at the


Temperature: 0.1 

c ways. well so that i will can all the flits on the hobbits. or old first did in a chill. so ferniath was help that you, and frodo would not ask that you come to be for a long year to fire. but i had


Temperature: 0.8 

ced and sitting on the stars hidden called the light of winding. 'what did you make the simpl